In [1]:
import d3graph
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
conflicts_file_path = "../Dados/Ukraine_conflicts.csv"

df_conflicts = pd.read_csv(conflicts_file_path)

display(df_conflicts)

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,9415087,804,UKR64033,64033,15 July 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Ukraine (2019-),...,Melitopol,46.8489,35.3653,1,061.ua,Subnational,"On 15 July 2022, Ukrainian forces shelled Meli...",0,1658249347,UKR
1,9415107,804,UKR63274,63274,15 July 2022,2022,1,Strategic developments,Disrupted weapons use,Military Forces of Ukraine (2019-),...,Bila Tserkva,49.8094,30.1121,2,24 Channel,National,"Interception: On 15 July 2022, Ukrainian air d...",0,1658249347,UKR
2,9415126,804,UKR63181,63181,15 July 2022,2022,1,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,...,Dnipro,48.4593,35.0386,1,24 Channel,National,"On 15 July 2022, Russian aircraft over the Cas...",0,1658249347,UKR
3,9415131,804,UKR63344,63344,15 July 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),...,Slovyansk,48.8533,37.6065,1,24 Channel,National,"On 15 July 2022, Russian forces shelled Slovya...",1,1658249347,UKR
4,9415142,804,UKR63275,63275,15 July 2022,2022,1,Strategic developments,Disrupted weapons use,Military Forces of Ukraine (2019-),...,Odesa,46.4639,30.7386,3,24 Channel,National,"Interception: On 15 July 2022, Ukrainian air d...",0,1658249347,UKR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39386,9092833,804,UKR23175,23175,16 July 2019,2019,1,Violence against civilians,Attack,Military Forces of Ukraine (2014-2019),...,Odesa,46.4639,30.7386,1,24 Channel; Novoye Vremya Ukraine,National,"On 16 July 2019, a man was shot in his leg by ...",0,1649875489,UKR
39387,9092834,804,UKR23176,23176,16 July 2019,2019,1,Protests,Peaceful protest,Protesters (Ukraine),...,Kyiv - Shevchenkivskyi,50.4646,30.4655,1,Novoye Vremya Ukraine; Right Sector; Svoboda; ...,Other-National,"On 16 July 2019, several dozen people, includi...",0,1649875489,UKR
39388,9092835,804,UKR23560,23560,16 July 2019,2019,1,Protests,Peaceful protest,Protesters (Ukraine),...,Odesa,46.4639,30.7386,1,Sayt Goroda Odessy,Subnational,"On 16 July 2019, people attended a demonstrati...",0,1649875489,UKR
39389,9175245,804,UKR23178,23178,16 July 2019,2019,1,Battles,Armed clash,NAF: United Armed Forces of Novorossiya,...,Novotroitske,47.7158,37.5919,2,JFO HQ press centre; Ministry of Defence of Uk...,Other,"On 16 July 2019, the NAF rebel forces fired gr...",0,1651165035,UKR


In [3]:
dfs = df_conflicts[df_conflicts['year'] >= 2022]

dfs = dfs[['event_date', 'event_type', 'sub_event_type', 'actor1', 'assoc_actor_1',
       'inter1', 'actor2', 'assoc_actor_2', 'inter2', 'interaction', 'admin1',
       'admin2', 'admin3', 'location', 'latitude', 'longitude', 'fatalities',
       'timestamp']]

dfs.head()

,event_date,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,admin1,admin2,admin3,location,latitude,longitude,fatalities,timestamp
0,15 July 2022,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Ukraine (2019-),NaN,1,NaN,NaN,0,10,Zaporizhia,Melitopolskyi,Melitopolska,Melitopol,46.8489,35.3653,0,1658249347
1,15 July 2022,Strategic developments,Disrupted weapons use,Military Forces of Ukraine (2019-),NaN,1,Military Forces of Russia (2000-),NaN,8,18,Kyiv,Bilotserkivskyi,Bilotserkivska,Bila Tserkva,49.8094,30.1121,0,1658249347
2,15 July 2022,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,NaN,8,NaN,NaN,0,80,Dnipropetrovsk,Dniprovskyi,Dniprovska,Dnipro,48.4593,35.0386,0,1658249347
3,15 July 2022,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),NaN,8,Civilians (Ukraine),NaN,7,78,Donetsk,Kramatorskyi,Slovianska,Slovyansk,48.8533,37.6065,1,1658249347
4,15 July 2022,Strategic developments,Disrupted weapons use,Military Forces of Ukraine (2019-),NaN,1,Military Forces of Russia (2000-),NaN,8,18,Odesa,Odeskyi,Odeska,Odesa,46.4639,30.7386,0,1658249347


In [4]:
cols_corr = dfs.columns

print(cols_corr)

Index(['event_date', 'event_type', 'sub_event_type', 'actor1', 'assoc_actor_1',
       'inter1', 'actor2', 'assoc_actor_2', 'inter2', 'interaction', 'admin1',
       'admin2', 'admin3', 'location', 'latitude', 'longitude', 'fatalities',
       'timestamp'],
      dtype='object')


In [5]:
import scipy.stats as ss
import seaborn as sns

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

confusion_matrix = pd.crosstab(dfs["sub_event_type"], dfs["event_type"])
cramers_v(confusion_matrix.values)

0.9993677829004163

In [6]:
sources = list()
targets = list()
weights = list()

cols = cols_corr

n = len(cols_corr)

for i in range(n):
    
    for j in range(n):
        sources.append(cols_corr[i])
        targets.append(cols_corr[j])

        val = pd.crosstab(dfs[cols_corr[i]], dfs[cols_corr[j]])

        weights.append(cramers_v(val.values)*100)
            

In [7]:
adjmat = d3graph.vec2adjmat(sources, targets, weight = weights)

In [8]:
print(adjmat)

target              actor1      actor2      admin1      admin2      admin3  \
source                                                                       
actor1          100.000000   35.873876   17.398168   22.200758   28.846934   
actor2           35.873876  100.000000   22.187551   29.942104   33.778437   
admin1           17.398168   22.187551  100.000000   99.667034   97.137274   
admin2           22.200758   29.942104   99.667034  100.000000   96.319409   
admin3           28.846934   33.778437   97.137274   96.319409  100.000000   
assoc_actor_1    82.305081   73.456426   40.736785   32.119883   39.718524   
assoc_actor_2    58.331180   89.088167   52.619561   43.502866   44.214584   
event_date       13.630307    9.009582   14.868930    9.235800    6.303237   
event_type       70.660171   48.230458   25.635652   37.667755   44.191670   
fatalities        0.000000   17.378371    4.035007    3.414056    8.914245   
inter1           99.880943   50.257472   23.272387   40.198435  

In [9]:
g = d3graph.d3graph()

In [25]:
g.graph(adjmat)

g.set_node_properties(color = 'cluster',
                      size = 10,
                      fontcolor = 'DarkBlue',
                      fontsize = 12
                     )

[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Number of unique nodes: 18
[d3graph] INFO> Number of unique nodes: 18


In [26]:
g.show(filepath = "../T2/Correlation_graph.html", 
       title = "<h1>Grafo de correlação entre atributos do dataset de conflitos<\h1>")

[d3graph] INFO> Slider range is set to [0, 100]
[d3graph] INFO> Write to path: [C:\Users\USER\Documents\Luca\PUC\5 Periodo\INF1032\Trabs\IntroDataScience-Inf1032\T2\Correlation_graph.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\USER\Documents\Luca\PUC\5 Periodo\INF1032\Trabs\IntroDataScience-Inf1032\T2\Correlation_graph.html]


In [12]:
print(help(g))

Help on d3graph in module d3graph.d3graph object:

class d3graph(builtins.object)
 |  d3graph(collision: float = 0.5, charge: int = 450, slider: List[int] = None, support='text', verbose: int = 20) -> None
 |  
 |  Create interactive networks in d3js.
 |  
 |  Description
 |  -----------
 |  d3graph is a Python library that is built on d3js to create interactive and standalone networks. The input
 |  data is an adjacency matrix for which the columns and indexes are the nodes and elements>0 the edges.
 |  The output is an HTML file that is interactive and standalone.
 |  
 |  Parameters
 |  ----------
 |  collision : float, (default: 0.5)
 |      Response of the network. Higher means that more collisions are prevented.
 |  charge : int, (default: 250)
 |      Edge length of the network. Towards zero becomes a dense network. Higher make edges longer.
 |  slider : list [min: int, max: int]:, (default: [None, None])
 |      [None, None] : Slider is automatically set to the min-max range us